In [21]:
from MasterThesis.classification import wandb_utils
import plotly.express as px

import wandb
wandb.login(key="3dfb06e1099f6465f340f29f3fbd268fac8216e2")

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/omar/.netrc


True

In [22]:
select_classes = ["non_agricultural", "agricultural_frontier"]

In [23]:
project = "Regression-2_classes"
entity = "omar-c"
name = "report"
version = ["RandomInit", "SimCLR", "Elevation+SimCLR","Elevation"]
train_size = [80, 100, 200, 500, 1000, 2000, 3000]


bar_plots = {}
for metric in ["test_total_loss"]:#["Recall", "Precision", "F1_score", "Acc_by_Class", "test_total_loss"]:
    plot = wandb_utils.lineplot_metrics_from_wandb(
        project=project,
        entity=entity,
        version=list(set(version)),
        train_size=list(set(train_size)),
        metric=f"{metric}",
        replace ={"SimCLR-v2": "SimCLR", "Elevation+SimCLR-v2": "Elevation+SimCLR", "RandomInit-v2":"RandomInit"},
    )

    bar_plots.update({metric: plot})

In [24]:
fig = bar_plots["test_total_loss"]
fig.update_layout(width=1200, height = 500)

In [28]:
def standar_report(project, entity, name, version, train_size, replace={}):

    bar_plots = {}
    for metric in ["F1_score"]:
        plot = wandb_utils.barplot_metrics_from_wandb(
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
        )

        bar_plots.update({metric: plot})

    line_plots = {}
    for metric in ["F1_score"]:  # ["Recall", "Precision", "F1_score", "Acc_by_Class"]:
        plot = wandb_utils.lineplot_metrics_from_wandb(
            project=project,
            entity=entity,
            version=list(set(version)),
            train_size=list(set(train_size)),
            metric=f"{metric}",
            replace=replace,
        )

        line_plots.update({metric: plot})

    # table = wandb_utils.get_table(
    #     project=project,
    #     entity=entity,
    #     version=version,
    #     train_size=train_size,
    # )

    # pr_curve = {}
    # for label in select_classes:

    #    plot = wandb_utils.plot_pr_curve(table.query(f"label=='{label}'"), color="train_size", line_dash="version")

    #    pr_curve.update({label: plot})

    # loss_table = wandb_utils.get_table(
    #    wandb,
    #    project=project,
    #    entity=entity,
    #    version=version,
    #    train_size=train_size,
    #    table_name="Loss",
    # )

    # loss_plot = wandb_utils.plot_loss_curves(loss_table, color="train_size", line_dash="version")

    with wandb.init(
        project="REPORT",
        entity=entity,
        config={"version": name, "train_size": None},
        name=name,
        resume=False,
        id=None,
    ) as run:

        # run.log({"Label Distribution": px.histogram(metadata, x="Classes", histnorm="percent")})
        [run.log({f"{name} Bareplot {metric}": plot}) for metric, plot in bar_plots.items()]
        [run.log({f"{name} Lineplot {metric}": plot}) for metric, plot in line_plots.items()]
        # [run.log({f"{name} PR Curve {label}": plot}) for label, plot in pr_curve.items()]
        # run.log({"Loss Plots": loss_plot})

In [26]:
standar_report(
    project="CLF-2_classes",
    entity="omar-c",
    name="CLF",
    version=["RandomInit-v2"] * 7 + ["SimCLR-v2"] * 7 + ["Elevation+SimCLR-v2"] * 7 + ["Elevation"] * 7,
    train_size=[80, 100, 200, 500, 1000, 2000, 3000] * 4,
    replace={"SimCLR-v2": "SimCLR", "Elevation+SimCLR-v2": "Elevation+SimCLR", "RandomInit-v2": "RandomInit"},
)

In [29]:
standar_report(
    project="Segmentation-2_classes",  # "Segmenation-2_classes",
    entity="omar-c",
    name="Segmentation-SimCLR",
    version=["RandomInit"] * 7 + ["SimCLR-v2"] * 7 + ["Elevation+SimCLR-v2"] * 7,
    train_size=[80, 100, 200, 500, 1000, 2000, 3000] * 3,
)

In [30]:
standar_report(
    project="Segmentation-2_classes",  # "Segmenation-2_classes",
    entity="omar-c",
    name="Segmentation-GLCNet",
    version=["RandomInit"] * 7 + ["GLCNet"] * 7 + ["Elevation+GLCNet"] * 7 + ["Elevation"] * 7,
    train_size=[80, 100, 200, 500, 1000, 2000, 3000] * 4,
)